In [11]:
# Dependencies
import requests
import json
import pandas as pd
import time
import io
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [12]:
# Source for the CSV Global Power Plant Database v1.3.0
# https://datasets.wri.org/dataset/globalpowerplantdatabase

# The path to our power plant CSV file
file = "../Resources/global_ppdb.csv"

# Read our power plant data into pandas
power_df = pd.read_csv(file, low_memory=False)

In [13]:
power_us_df = power_df.loc[power_df['country_long'] == 'United States of America'].reset_index(drop=True)
power_us_df

,country,country_long,name,gppd_idnr,capacity_mw,latitude,longitude,primary_fuel,other_fuel1,other_fuel2,...,estimated_generation_gwh_2013,estimated_generation_gwh_2014,estimated_generation_gwh_2015,estimated_generation_gwh_2016,estimated_generation_gwh_2017,estimated_generation_note_2013,estimated_generation_note_2014,estimated_generation_note_2015,estimated_generation_note_2016,estimated_generation_note_2017
0,USA,United States of America,100 Brook Hill Drive Solar,USA0063292,2.0,41.0930,-73.9828,Solar,NaN,NaN,...,NaN,NaN,NaN,NaN,3.25,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,CAPACITY-FACTOR-V1
1,USA,United States of America,1025 Traveller Solar LLC,USA0062660,5.0,35.4273,-79.1263,Solar,NaN,NaN,...,NaN,NaN,NaN,NaN,8.14,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,CAPACITY-FACTOR-V1
2,USA,United States of America,1047 Little Mountain Solar LLC,USA0062661,3.0,36.1971,-80.8067,Solar,NaN,NaN,...,NaN,NaN,NaN,NaN,4.88,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,CAPACITY-FACTOR-V1
3,USA,United States of America,12 Applegate Solar LLC,USA0059371,1.9,40.2003,-74.5761,Solar,NaN,NaN,...,2.92,2.94,3.06,2.85,2.61,SOLAR-V1,SOLAR-V1,SOLAR-V1,SOLAR-V1,SOLAR-V1
4,USA,United States of America,126 Grove Solar LLC,USA0060858,2.0,42.0761,-71.4227,Solar,NaN,NaN,...,3.01,2.98,3.11,2.98,2.64,SOLAR-V1,SOLAR-V1,SOLAR-V1,SOLAR-V1,SOLAR-V1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9828,USA,United States of America,Zion Landfill Gas to Energy Facility,USA0056871,7.0,42.4803,-87.8861,Waste,NaN,NaN,...,NaN,NaN,NaN,NaN,1.26,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,CAPACITY-FACTOR-V1
9829,USA,United States of America,Zorn,USA0001368,18.0,38.2803,-85.7023,Gas,NaN,NaN,...,NaN,NaN,NaN,NaN,63.47,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,CAPACITY-FACTOR-V1
9830,USA,United States of America,Zotos International WPGF,USA0057648,3.4,42.8869,-76.9683,Wind,NaN,NaN,...,8.31,8.46,6.57,6.52,6.81,WIND-V1,WIND-V1,WIND-V1,WIND-V1,WIND-V1
9831,USA,United States of America,Zumbro Community Solar Garden,USA0061574,1.0,44.3195,-92.6703,Solar,NaN,NaN,...,NaN,NaN,NaN,NaN,1.62,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,CAPACITY-FACTOR-V1


In [14]:
# Checking for any efficiencies gained from repeated values - insignificant
power_us_df[['latitude', 'longitude']].nunique()

latitude     9271
longitude    9519
dtype: int64

In [15]:
# Create batches based on iloc ranges because I had issues running this for 3 hours straight.
# There was also guidance and policy usage restriction for bulk usage, so this allowed me to break up the data gathering.
# I ran these one at a time and saved to CSV
# Separated calls to Nominatimin in batches

power_us_batch_df = power_us_df.iloc[:2000, :]    
# power_us_batch_df = power_us_df.iloc[2000:4000, :]   
# power_us_batch_df = power_us_df.iloc[4000:6000, :]   
# power_us_batch_df = power_us_df.iloc[6000:8000, :]  
# power_us_batch_df = power_us_df.iloc[8000:, :]
power_us_batch_df

,country,country_long,name,gppd_idnr,capacity_mw,latitude,longitude,primary_fuel,other_fuel1,other_fuel2,...,estimated_generation_gwh_2013,estimated_generation_gwh_2014,estimated_generation_gwh_2015,estimated_generation_gwh_2016,estimated_generation_gwh_2017,estimated_generation_note_2013,estimated_generation_note_2014,estimated_generation_note_2015,estimated_generation_note_2016,estimated_generation_note_2017
0,USA,United States of America,100 Brook Hill Drive Solar,USA0063292,2.0,41.0930,-73.9828,Solar,NaN,NaN,...,NaN,NaN,NaN,NaN,3.25,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,CAPACITY-FACTOR-V1
1,USA,United States of America,1025 Traveller Solar LLC,USA0062660,5.0,35.4273,-79.1263,Solar,NaN,NaN,...,NaN,NaN,NaN,NaN,8.14,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,CAPACITY-FACTOR-V1
2,USA,United States of America,1047 Little Mountain Solar LLC,USA0062661,3.0,36.1971,-80.8067,Solar,NaN,NaN,...,NaN,NaN,NaN,NaN,4.88,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,CAPACITY-FACTOR-V1
3,USA,United States of America,12 Applegate Solar LLC,USA0059371,1.9,40.2003,-74.5761,Solar,NaN,NaN,...,2.92,2.94,3.06,2.85,2.61,SOLAR-V1,SOLAR-V1,SOLAR-V1,SOLAR-V1,SOLAR-V1
4,USA,United States of America,126 Grove Solar LLC,USA0060858,2.0,42.0761,-71.4227,Solar,NaN,NaN,...,3.01,2.98,3.11,2.98,2.64,SOLAR-V1,SOLAR-V1,SOLAR-V1,SOLAR-V1,SOLAR-V1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,USA,United States of America,Coronus Adelanto West 1,USA0059536,1.5,34.5481,-117.4644,Solar,NaN,NaN,...,NaN,NaN,NaN,3.25,3.63,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,SOLAR-V1,SOLAR-V1
1996,USA,United States of America,Coronus Adelanto West 2,USA0059537,1.5,34.5481,-117.4644,Solar,NaN,NaN,...,NaN,NaN,NaN,3.25,3.63,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,SOLAR-V1,SOLAR-V1
1997,USA,United States of America,Corpus Christi,USA0050475,41.0,27.8111,-97.5958,Gas,NaN,NaN,...,NaN,NaN,NaN,NaN,144.58,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,CAPACITY-FACTOR-V1
1998,USA,United States of America,Corpus Christi Energy Center,USA0055206,593.3,27.8139,-97.4283,Gas,NaN,NaN,...,NaN,NaN,NaN,NaN,2092.26,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,CAPACITY-FACTOR-V1


In [16]:
# The function city_state_county(row) takes a row of the DataFrame as input and extracts
# the coordinates of the location from the latitude and longitude columns. 
# The geolocator.reverse() method is used to reverse geocode the coordinates and obtain
# the location information as a JSON-like dictionary in the address variable.
# The function then extracts the city, state, town, county, and zip code information
# from the address dictionary using the get() method and assigns them to the respective columns
# of the DataFrame row. The function also includes a time.sleep() call with an argument of 1
# to pause the execution for 1 second between each row to avoid overloading the geocoding service with too many requests.

# Finally, the city_state_county() function is applied to each row of the DataFrame using
# the apply() method with the axis=1 parameter to apply the function row-wise. 
# The function returns the modified row, which is assigned back to the original DataFrame.

# I used this exaple as a template
# https://stackoverflow.com/questions/69409255/how-to-get-city-state-and-country-from-a-list-of-latitude-and-longitude-coordi

# Get county, state and zip code for all the power stations in the US

# bounds = [[0, 2000], [2000, 4000], [4000, 6000], [6000, 8000], [80, power_us_batch_df.shape[0]]]
bounds = [[0, 2], [2, 4], [4, 6], [6, 8], [8, 10]]

final_power_df = pd.DataFrame()

geolocator = Nominatim(user_agent="world_power")
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)

def city_state_county(row):
    coord = f"{row['latitude']}, {row['longitude']}"
    location = reverse(coord, exactly_one=True)
    address = location.raw['address']
    city = address.get('city', '')
    state = address.get('state', '')
    town = address.get('town', '')
    county = address.get('county', '')
    zipcode = address.get('postcode', '')
    row['city'] = city
    row['town'] = town
    row['state'] = state
    row['county'] = county
    row['postcode'] = zipcode
    # time.sleep(1)
    return row

for b in bounds:
    temp_df = power_us_batch_df.iloc[b[0]:b[1], :]
    temp_df_address = temp_df.apply(city_state_county, axis=1)
    final_power_df = pd.concat([final_power_df, temp_df_address])
    time.sleep(2)

final_power_df

,country,country_long,name,gppd_idnr,capacity_mw,latitude,longitude,primary_fuel,other_fuel1,other_fuel2,...,estimated_generation_note_2013,estimated_generation_note_2014,estimated_generation_note_2015,estimated_generation_note_2016,estimated_generation_note_2017,city,town,state,county,postcode
0,USA,United States of America,100 Brook Hill Drive Solar,USA0063292,2.0,41.0930,-73.9828,Solar,NaN,NaN,...,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,CAPACITY-FACTOR-V1,,,New York,Rockland County,10994
1,USA,United States of America,1025 Traveller Solar LLC,USA0062660,5.0,35.4273,-79.1263,Solar,NaN,NaN,...,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,CAPACITY-FACTOR-V1,,Sanford,North Carolina,Lee County,27332
2,USA,United States of America,1047 Little Mountain Solar LLC,USA0062661,3.0,36.1971,-80.8067,Solar,NaN,NaN,...,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,CAPACITY-FACTOR-V1,,,North Carolina,Yadkin County,28642
3,USA,United States of America,12 Applegate Solar LLC,USA0059371,1.9,40.2003,-74.5761,Solar,NaN,NaN,...,SOLAR-V1,SOLAR-V1,SOLAR-V1,SOLAR-V1,SOLAR-V1,,,New Jersey,Mercer County,08691
4,USA,United States of America,126 Grove Solar LLC,USA0060858,2.0,42.0761,-71.4227,Solar,NaN,NaN,...,SOLAR-V1,SOLAR-V1,SOLAR-V1,SOLAR-V1,SOLAR-V1,,Franklin,Massachusetts,Norfolk County,02038
5,USA,United States of America,1420 Coil Av #C,USA0057310,1.3,33.7943,-118.2414,Solar,NaN,NaN,...,SOLAR-V1,SOLAR-V1,SOLAR-V1,SOLAR-V1,SOLAR-V1,Los Angeles,,California,Los Angeles County,90744
6,USA,United States of America,145 Talmadge Solar,USA0057458,3.8,40.5358,-74.3913,Solar,NaN,NaN,...,SOLAR-V1,SOLAR-V1,SOLAR-V1,SOLAR-V1,SOLAR-V1,,Edison,New Jersey,Middlesex County,08817
7,USA,United States of America,1515 S Caron Road,USA0007770,4.2,41.9084,-89.0466,Gas,NaN,NaN,...,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,NO-ESTIMATION,CAPACITY-FACTOR-V1,,Rochelle,Illinois,Ogle County,61068
8,USA,United States of America,158th Fighter Wing Solar Farm,USA0060542,1.3,44.4777,-73.1534,Solar,NaN,NaN,...,SOLAR-V1,SOLAR-V1,SOLAR-V1,SOLAR-V1,SOLAR-V1,,South Burlington,Vermont,Chittenden County,05403
9,USA,United States of America,180 Raritan Solar,USA0058187,1.9,40.5161,-74.3400,Solar,NaN,NaN,...,SOLAR-V1,SOLAR-V1,SOLAR-V1,SOLAR-V1,SOLAR-V1,,Edison,New Jersey,Middlesex County,08837


In [ ]:
# The path to our CSV file
file = "Output/final_power_us_na.csv"

# Read our power plant data into pandas that we save after every succesful batch pull to concat with current batch pull
# This is not run on the first batch
rec_final_df = pd.read_csv(file)
rec_final_df

,Unnamed: 0,primary_fuel,state,county,postcode
0,0,Solar,New York,Rockland County,10994.0
1,1,Solar,North Carolina,Lee County,27332.0
2,2,Solar,North Carolina,Yadkin County,28642.0
3,3,Solar,New Jersey,Mercer County,8691.0
4,4,Solar,Massachusetts,Norfolk County,2038.0
...,...,...,...,...,...
9828,9828,Waste,Illinois,Lake County,60096.0
9829,9829,Gas,Kentucky,Jefferson County,40207.0
9830,9830,Wind,New York,Ontario County,14456.0
9831,9831,Solar,Minnesota,Goodhue County,55992.0


In [18]:
# Concat current batch with previously saved batch and pulled back in the DF
# This is not run on the first batch
comp_power_df = [rec_final_df, power_us_b4_df]
final_power_df = pd.concat(comp_power_df)

final_power_df

NameError: name 'rec_final_df' is not defined

In [17]:
# More data cleaning with Dropna
final_power_na_df = final_power_df.dropna(axis = 0, how ='any')
final_power_na_df

,primary_fuel,state,county
0,Solar,New York,Rockland County
1,Solar,North Carolina,Lee County
2,Solar,North Carolina,Yadkin County
3,Solar,New Jersey,Mercer County
4,Solar,Massachusetts,Norfolk County
...,...,...,...
9828,Waste,Illinois,Lake County
9829,Gas,Kentucky,Jefferson County
9830,Wind,New York,Ontario County
9831,Solar,Minnesota,Goodhue County


In [10]:
# Remove unwanted columns
final_power_na_df = final_power_df[['primary_fuel', 'state', 'county', 'postcode']]


NameError: name 'final_power_df' is not defined

In [18]:
# Save DF to CSV file. This was ran after every data collection of each batch.
final_power_na_df.to_csv("Output/final_power_us_na.csv")